In [1]:
import pandas as pd
import numpy as np
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import quadratic_weighted_kappa
import feature_generator
from pybrain.structure import FeedForwardNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection
from pybrain.structure import BiasUnit

In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    

In [3]:
features = feature_generator.GetFeatures(dfTrain, dfTest, 100)

Medical_History_2
Medical_History_10


In [4]:
ds = SupervisedDataSet(len(features), 8)

In [5]:
ds['input'], ds['target']

(array([], shape=(0, 327), dtype=float64),
 array([], shape=(0, 8), dtype=float64))

In [6]:
# target = dfTrain['Response'].values
target = np.zeros((len(dfTrain), 8))
for i in range(len(dfTrain)):
        target[i,dfTrain.iloc[i]['Response'] - 1] = 1
#         target[i] = dfTrain.iloc[i]['Response']

In [7]:
for i in range(len(dfTrain)):
    ds.addSample(dfTrain.iloc[i][features].values, target[i])

In [22]:
net = buildNetwork(len(features),64,8, bias=True)

# net = FeedForwardNetwork()
# inLayer = LinearLayer(len(features))
# hiddenLayer1 = SigmoidLayer(128)
# hiddenLayer2 = SigmoidLayer(64)
# outLayer = LinearLayer(8)

# net.addInputModule(inLayer)
# net.addModule(hiddenLayer1)
# net.addModule(hiddenLayer2)
# net.addOutputModule(outLayer)
# net.addModule(BiasUnit(name = 'bias'))

# in_to_hidden = FullConnection(inLayer, hiddenLayer1)
# hidden1_to_hidden2 = FullConnection(hiddenLayer1, hiddenLayer2)
# hidden_to_out = FullConnection(hiddenLayer2, outLayer)

# net.addConnection(in_to_hidden)
# # net.addConnection(FullConnection(net['bias'], hiddenLayer1))
# # net.addConnection(FullConnection(net['bias'], hiddenLayer2))
# net.addConnection(hidden1_to_hidden2)
# net.addConnection(hidden_to_out)

# net.sortModules()

trainer = BackpropTrainer(net, ds, learningrate=0.01, momentum=0.05, verbose=True)

In [23]:
# error = trainer.train()
# errors = [error,]
trainer.trainUntilConvergence(maxEpochs = 5)
# for iterations in range(3):
#     print iterations, errors[-1]
#     errors.append(trainer.train())

Total error: 0.0689725681423
Total error: 0.0576597263429
Total error: 0.0568761265576
Total error: 0.0566508699597
Total error: 0.0565277223072
Total error: 0.0562112673956
train-errors: [  0.068973  0.057660  0.056876  0.056651  0.056528  0.056211  0.051451]
valid-errors: [  4.213695  0.052472  0.054363  0.051966  0.061029  0.056644  0.054452]


([0.068972568142296478,
  0.057659726342945326,
  0.056876126557550359,
  0.056650869959709027,
  0.056527722307169334,
  0.056211267395612295,
  0.051450896338997382],
 [4.2136953277920197,
  0.05247234981623667,
  0.054363243175196904,
  0.051965572306197354,
  0.061028731841472769,
  0.056643506227481502,
  0.054452298765444787])

In [24]:
predictions = trainer.testOnClassData()

In [25]:
predictions = np.asarray(predictions) + 1
# predictions = np.clip(np.asarray(predictions), 1, 8)


In [26]:
print predictions

[8 4 8 ..., 8 8 8]


In [27]:
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain['Response'])

0.11163437123
